<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/toy_self_driving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Perception: Object Detection Model
def build_object_detection_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')  # 10 classes for objects
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prediction: Trajectory Prediction Model
def build_trajectory_prediction_model():
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=(5,)),  # 5 inputs (e.g., position, velocity)
        layers.Dense(128, activation='relu'),
        layers.Dense(2)  # Outputs: future x, y coordinates
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Planning: Path Planning Model
def build_path_planning_model():
    model = models.Sequential([
        layers.LSTM(64, return_sequences=True, input_shape=(10, 2)),  # Sequence of waypoints
        layers.LSTM(64),
        layers.Dense(10)  # Output: 10 control signals
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Create toy datasets for each model
def create_toy_data():
    x_perception = np.random.rand(100, 64, 64, 3)  # 100 images of 64x64 with 3 channels
    y_perception = tf.keras.utils.to_categorical(np.random.randint(0, 10, 100), 10)
    x_prediction = np.random.rand(100, 5)  # 100 samples of 5 features
    y_prediction = np.random.rand(100, 2)  # 100 samples of 2 outputs (x, y)
    x_planning = np.random.rand(100, 10, 2)  # 100 sequences of 10 waypoints with 2D coords
    y_planning = np.random.rand(100, 10)  # 100 sequences of control signals
    return (x_perception, y_perception), (x_prediction, y_prediction), (x_planning, y_planning)

# Build and train the models
(x_perception, y_perception), (x_prediction, y_prediction), (x_planning, y_planning) = create_toy_data()

object_detection_model = build_object_detection_model((64, 64, 3))
object_detection_model.fit(x_perception, y_perception, epochs=5, batch_size=10)

trajectory_prediction_model = build_trajectory_prediction_model()
trajectory_prediction_model.fit(x_prediction, y_prediction, epochs=5, batch_size=10)

path_planning_model = build_path_planning_model()
path_planning_model.fit(x_planning, y_planning, epochs=5, batch_size=10)

# Example pipeline integration
def autonomous_pipeline(image, vehicle_state, waypoints):
    detected_objects = object_detection_model.predict(np.expand_dims(image, axis=0))
    predicted_trajectory = trajectory_prediction_model.predict(vehicle_state)
    planned_path = path_planning_model.predict(np.expand_dims(waypoints, axis=0))
    return detected_objects, predicted_trajectory, planned_path

# Test the pipeline
test_image = np.random.rand(64, 64, 3)
test_vehicle_state = np.random.rand(1, 5)
test_waypoints = np.random.rand(1, 10, 2)
detected_objects, predicted_trajectory, planned_path = autonomous_pipeline(test_image, test_vehicle_state, test_waypoints)

print("Detected Objects:", detected_objects)
print("Predicted Trajectory:", predicted_trajectory)
print("Planned Path:", planned_path)
